# pkpd-tutorial
This notebook shows some simple use cases of PKPD models.
The tutorial must work in a minimal environment consisting of
* `numpy`
* `scipy`
* `matplotlib`

For questions contact konigmatt@googlemail.com. The latest version of this tutorial is available from github at https://github.com/matthiaskoenig/pkpd-tutorial

In [7]:
from __future__ import print_function
import numpy as np
import scipy
from matplotlib import pylab as plt

In [4]:
# load the example model





In [5]:
# simulate the example model



In [6]:
# plot the results